In [1]:
import pandas as pd

### Data Processing and Analysis

In [27]:
data = pd.read_csv('Customer Churn.csv')

# changing status column from 1: Active, 2: In-Active to 0 and 1
data['Status'] = data['Status'] - 1

# changing column name to represent 0: False (active) and 1: True (inactive)
# Renaming other columns to make them consistent - removing 2 space between words, upper casing subsequent words
data.rename(columns={'Status': 'Inactivity', 'Call  Failure':'Call Failure', 
                     'Subscription  Length':'Subscription Length', 
                     'Charge  Amount':'Charge Amount', 'Frequency of use':'Frequency of Use'}, inplace=True)

# Caluclating the call failure rate and adding it as a column 
data['Call Failure Rate'] = data['Call Failure'] / data['Frequency of Use']
data

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of Use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Inactivity,Age,Customer Value,Churn,Call Failure Rate
0,8,0,38,0,4370,71,5,17,3,1,0,30,197.640,0,0.112676
1,0,0,39,0,318,5,7,4,2,1,1,25,46.035,0,0.000000
2,10,0,37,0,2453,60,359,24,3,1,0,30,1536.520,0,0.166667
3,10,0,38,0,4198,66,1,35,1,1,0,15,240.020,0,0.151515
4,3,0,38,0,2393,58,2,33,1,1,0,15,145.805,0,0.051724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,21,0,19,2,6697,147,92,44,2,2,0,25,721.980,0,0.142857
3146,17,0,17,1,9237,177,80,42,5,1,0,55,261.210,0,0.096045
3147,13,0,18,4,3157,51,38,21,3,1,0,30,280.320,0,0.254902
3148,7,0,11,2,4695,46,222,12,3,1,0,30,1077.640,0,0.152174


To Do:
1. Summary Statistics of each column
2. outliers / null / empty
3. balance of churn
4. Graphs of data (distribution of each column and then fit something)

Initial analysis
6. PCA / Heirachal clustering
7. lasso/ridge in the classification to see which predictors are important

Classification
1. random forest
2. boosted model

Regression
1. logistic regression